In [ ]:
import numpy as np
import wandb
from matplotlib import pyplot as plt
from util import populate_plt_settings, get_column_width, get_fig_size, get_latex_float
import re
import plotly.express as px
import pandas as pd

In [ ]:
populate_plt_settings(plt)

In [ ]:
api = wandb.Api()
# run, target_step = api.run('kennychufk/alluvion-rl/fa8dq4m1Aug-diagonal100'), 600
# run, target_step = api.run('kennychufk/alluvion-rl/fa8dq4m1Aug-val-diagonal2-0.011'), 600

# run, target_step = api.run('kennychufk/alluvion-rl/2v7m4mucAug-val-loop2-0.011'), 2600 
# run, target_step = api.run('kennychufk/alluvion-rl/fa8dq4m1Aug-val-loop2-0.011'), 600 
# run, target_step = api.run('kennychufk/alluvion-rl/2nw7bxr4Aug-val-loop2-0.011'), 600 

# run, target_step = api.run('kennychufk/alluvion-rl/2v7m4mucAug-val-bidir-circles2-0.011'), 2600
# run, target_step = api.run('kennychufk/alluvion-rl/fa8dq4m1Aug-val-bidir-circles2-0.011'), 600
# run, target_step = api.run('kennychufk/alluvion-rl/2nw7bxr4Aug-val-bidir-circles2-0.011'), 600

# run, target_step = api.run('kennychufk/alluvion-rl/2v7m4mucAug-val-nephroid2-0.011'), 2600
# run, target_step = api.run('kennychufk/alluvion-rl/fa8dq4m1Aug-val-nephroid2-0.011'), 600
# run, target_step = api.run('kennychufk/alluvion-rl/2nw7bxr4Aug-val-nephroid2-0.011'), 600

# run, target_step = api.run('kennychufk/alluvion-rl/2v7m4mucAug-val-diagonal2-0.011'), 2600
# run, target_step = api.run('kennychufk/alluvion-rl/fa8dq4m1Aug-val-diagonal2-0.011'), 600
# run, target_step = api.run('kennychufk/alluvion-rl/2nw7bxr4Aug-val-diagonal2-0.011'), 600

In [ ]:
def get_grouped_scores(run, target_step, metric_name):
    history = run.scan_history(keys=None,
                           page_size=1000,
                           min_step=None,
                           max_step=None)
    pattern=re.compile('([0-9]+)-uth-([0-9a-zA-z]+)-'+metric_name+'-m%')

    buoy_counts_in_trials = []

    step_interval = 50
    for row in history:
        if row['_step']+1!=target_step:
            continue
        for key in row:
            match_result = pattern.match(key)
            if match_result is not None:
                num_buoys = int(match_result[1])
                buoy_counts_in_trials.append(num_buoys)
        break
    buoy_counts_in_trials.sort()

    buoy_counts_in_trials
    buoy_counts_grouped = []
    curr_buoy_count = -1
    curr_idx =-1
    for buoy_count in buoy_counts_in_trials:
        if curr_buoy_count!=buoy_count:
            buoy_counts_grouped.append([])
            curr_buoy_count = buoy_count
            curr_idx+=1
        buoy_counts_grouped[curr_idx].append(buoy_count)

    buoy_count_to_id = {}
    for idx, buoy_counts_same in enumerate(buoy_counts_grouped):
        buoy_count_to_id[buoy_counts_same[0]] = idx


    val_scores = []
    for buoy_counts_same in buoy_counts_grouped:
        val_scores.append(np.zeros(len(buoy_counts_same)))
    val_scores_named = []


    cursors = np.zeros(len(buoy_counts_grouped), int)
    for row_id, row in enumerate(history):
        if (row['_step']+1)%50!=0:
            continue
        if row['_step']+1!=target_step:
            continue
        for key in row:
            match_result = pattern.match(key)
            if match_result is not None:
                num_buoys = int(match_result[1])
                buoy_count_index = buoy_count_to_id[num_buoys]
                score = 1-row[key]
                val_scores[buoy_count_index][cursors[buoy_count_index]] = score
                val_scores_named.append([f'{num_buoys:03d}-{match_result[2]}', score])
                val_scores
                cursors[buoy_count_index]+=1
        break
    distinct_buoy_counts = []
    for buoy_counts_same in buoy_counts_grouped:
        distinct_buoy_counts.append(buoy_counts_same[0])
    val_scores_named.sort(key=lambda x: x[0])
    
    return val_scores, val_scores_named, buoy_counts_grouped, distinct_buoy_counts

def print_scores(val_scores_named):
    for name_score in val_scores_named:
        print(f'{name_score[0]},{name_score[1]}')
        
def print_buoy_statistics(distinct_buoy_counts, score_mean, score_std):
    for i, buoy_count in enumerate(distinct_buoy_counts):
        print(f'{buoy_count},{score_mean[i]},{score_std[i]}')
        
def get_statistics(val_scores):
    score_std = np.zeros(len(val_scores))
    score_mean = np.zeros(len(val_scores))

    for i, scores in enumerate(val_scores):
        score_std[i] = scores.std()
        score_mean[i] = scores.mean()
    return score_mean, score_std

In [ ]:
homo_eulerian_scores, homo_eulerian_scores_named, homo_eulerian_buoy_counts_grouped, eulerian_distinct_buoy_counts = get_grouped_scores(api.run('kennychufk/alluvion-rl/fa8dq4m1Aug-diagonal100'), 600, 'eulerian_masked')
# homo_height_field_scores, homo_height_field_scores_named, homo_height_field_buoy_counts_grouped, height_field_distinct_buoy_counts = get_grouped_scores(api.run('kennychufk/alluvion-rl/fa8dq4m1Aug-val-diagonal2-0.011'), 600, 'height_field')
hetero_eulerian_scores, hetero_eulerian_scores_named, hetero_eulerian_buoy_counts_grouped, eulerian_distinct_buoy_counts = get_grouped_scores(api.run('kennychufk/alluvion-rl/2v7m4mucAug-diagonal100'), 2600, 'eulerian_masked')
# hetero_height_field_scores, hetero_height_field_scores_named, hetero_height_field_buoy_counts_grouped, height_field_distinct_buoy_counts = get_grouped_scores(api.run('kennychufk/alluvion-rl/2v7m4mucAug-val-diagonal2-0.011'), 2600, 'height_field')

In [ ]:
for i in range(len(homo_eulerian_buoy_counts_grouped)):
    plt.scatter(homo_eulerian_buoy_counts_grouped[i], homo_eulerian_scores[i])

In [ ]:
homo_eulerian_mean, homo_eulerian_std = get_statistics(homo_eulerian_scores)
# homo_height_field_mean, homo_height_field_std = get_statistics(homo_height_field_scores)

hetero_eulerian_mean, hetero_eulerian_std = get_statistics(hetero_eulerian_scores)
# hetero_height_field_mean, hetero_height_field_std = get_statistics(hetero_height_field_scores)

In [ ]:
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
num_rows = 1
num_cols = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize = get_fig_size(get_column_width(), ratio=(np.sqrt(5)-1)*0.5))

ax.errorbar(eulerian_distinct_buoy_counts, hetero_eulerian_mean, yerr=hetero_eulerian_std, capsize=2, capthick=1, label=r'Trained with \emph{Stars \& trefoiloids} (Fig. 9b)')
ax.errorbar(eulerian_distinct_buoy_counts, homo_eulerian_mean, yerr=homo_eulerian_std, capsize=2, capthick=1, label=r'Trained with \emph{Diagonal oscillations} (Fig. 9a)')
# ax.errorbar(height_field_distinct_buoy_counts, homo_height_field_mean, yerr=homo_height_field_std, capsize=2, capthick=1, label='Trained height field')
# ax.errorbar(height_field_distinct_buoy_counts, hetero_height_field_mean, yerr=hetero_height_field_std, capsize=2, capthick=1, label='Validated height field')
ax.set_xlabel('Number of buoys')
ax.set_ylabel('Eulerian score')

ax.set_xlim(0,100)
ax.xaxis.set_major_locator(MultipleLocator(10))
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.grid(which='minor', color='#DDDDDD', linewidth=0.5)
ax.legend()

fig.tight_layout(pad=0.05) # should set tight_layout before add_axes()
fig.savefig('buoy-count-score-correlation.pgf')